# Naver blog scrap

In [2]:
first_list = [ [] ] * 10
len(first_list), first_list

(10, [[], [], [], [], [], [], [], [], [], []])

In [3]:
date_list = first_list[0]
id(date_list)

2191901671232

In [5]:
rare_number_list  = first_list[1]
id(rare_number_list )

2191901671232

In [6]:
date_list.append(1)
date_list, rare_number_list

([1], [1])

## 라이브러리

In [32]:
import pandas as pd
import time, os, random
from datetime import datetime
from numpy import nan

In [33]:
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import chromedriver_autoinstaller as ca

## 현재 파일 위치 확인

In [34]:
# 현재 파일 위치 확인
code_path = os.getcwd().replace('\\', '/')
code_path

'c:/data/Wantreez/Crawling/incheon'

## 수집한 파일 저장 폴더 생성

In [35]:
# 수집한 파일을 저장할 폴더 생성
crawled_folder_path = code_path + '/crawled_data/naver_blog'
os.makedirs(crawled_folder_path, exist_ok=True)

## 크롬 드라이버 버전 확인 및 설치

In [36]:
# 크롬 드라이버 버전 확인
chrome_ver = ca.get_chrome_version().split('.')[0]
chrome_ver

'100'

In [37]:
# 크롬 드라이버 설치 확인
try:
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')
except:
    ca.install(True)
    driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/3261653714.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(code_path + f'/{chrome_ver}/' + 'chromedriver.exe')


## 페이지 접속

In [38]:
# 페이지 접속
url = 'https://blog.naver.com/PostView.naver?blogId=icouncil103&logNo=222657505998&categoryNo=9&parentCategoryNo=9&from=thumbnailList'
driver.get(url)
driver.implicitly_wait(3)
driver.maximize_window()

In [39]:
# page 정보
page_info = url.split('=')[2].split('&')[0]
page_info

'222657505998'

## 페이지 스크래핑

### 댓글 버튼 클릭

In [40]:
# 댓글 버튼 클릭
comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')
comment_button.click()
time.sleep(1)

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/3197372586.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  comment_button = driver.find_element_by_xpath(f'//*[@id="Comi{page_info}"]')


### 이전 페이지 이동

In [41]:
# 이전 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/2309467251.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()


### 1페이지

In [42]:
# 1 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/593960153.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[2]').click()


In [43]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [46]:
date_list = []
rare_number_list = []
check_reply_list = []
nickname_list = []
writer_url_list = []
like_num_list = []
link_num_list = []
reply_num_list = []
img_list = []
comment_list = []

for one in comment_areas:
# 작성일 수집
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호

    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분

    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임

    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url

    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수

    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수

    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수

    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)


# 첨부이미지url

    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)


# 댓글 내용 수집

    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [47]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  50
고유번호 리스트 길이 :  50
답글구분 리스트 길이 :  50
닉네임 리스트 길이 :  50
작성자URL 리스트 길이 :  50
공감수 리스트 길이 :  50
링크수 리스트 길이 :  50
답글수 리스트 길이 :  50
첨부이미지URL 리스트 길이 :  50
내용 리스트 길이 :  50


### 2페이지

In [48]:
# 2 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[1]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/1784597422.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[1]').click()


In [49]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [50]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [51]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  100
고유번호 리스트 길이 :  100
답글구분 리스트 길이 :  100
닉네임 리스트 길이 :  100
작성자URL 리스트 길이 :  100
공감수 리스트 길이 :  100
링크수 리스트 길이 :  100
답글수 리스트 길이 :  100
첨부이미지URL 리스트 길이 :  100
내용 리스트 길이 :  100


### 3페이지

In [52]:
# 3 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/3873755998.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[3]').click()


In [53]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [54]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [55]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  150
고유번호 리스트 길이 :  150
답글구분 리스트 길이 :  150
닉네임 리스트 길이 :  150
작성자URL 리스트 길이 :  150
공감수 리스트 길이 :  150
링크수 리스트 길이 :  150
답글수 리스트 길이 :  150
첨부이미지URL 리스트 길이 :  150
내용 리스트 길이 :  150


### 4페이지

In [56]:
# 4 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/1449846845.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[4]').click()


In [57]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [58]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [59]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  200
고유번호 리스트 길이 :  200
답글구분 리스트 길이 :  200
닉네임 리스트 길이 :  200
작성자URL 리스트 길이 :  200
공감수 리스트 길이 :  200
링크수 리스트 길이 :  200
답글수 리스트 길이 :  200
첨부이미지URL 리스트 길이 :  200
내용 리스트 길이 :  200


### 5페이지

In [60]:
# 5 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/1452808398.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[5]').click()


In [61]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [62]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [63]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  250
고유번호 리스트 길이 :  250
답글구분 리스트 길이 :  250
닉네임 리스트 길이 :  250
작성자URL 리스트 길이 :  250
공감수 리스트 길이 :  250
링크수 리스트 길이 :  250
답글수 리스트 길이 :  250
첨부이미지URL 리스트 길이 :  250
내용 리스트 길이 :  250


### 6페이지

In [64]:
# 6 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/3074135804.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[6]').click()


In [65]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [66]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [67]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  300
고유번호 리스트 길이 :  300
답글구분 리스트 길이 :  300
닉네임 리스트 길이 :  300
작성자URL 리스트 길이 :  300
공감수 리스트 길이 :  300
링크수 리스트 길이 :  300
답글수 리스트 길이 :  300
첨부이미지URL 리스트 길이 :  300
내용 리스트 길이 :  300


### 7페이지

In [68]:
# 7 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[7]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_7724/499117945.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[7]').click()


In [69]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

51

In [70]:
date_list = []
rare_number_list = []
check_reply_list = []
nickname_list = []
writer_url_list = []
like_num_list = []
link_num_list = []
reply_num_list = []
img_list = []
comment_list = []

# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [71]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  51
고유번호 리스트 길이 :  51
답글구분 리스트 길이 :  51
닉네임 리스트 길이 :  51
작성자URL 리스트 길이 :  51
공감수 리스트 길이 :  51
링크수 리스트 길이 :  51
답글수 리스트 길이 :  51
첨부이미지URL 리스트 길이 :  51
내용 리스트 길이 :  51


### 7페이지 문제 확인 중...
* undefined 발생
    * 데이터 셋에서 제거

In [72]:
date_list[:10]

['2022-02-27 17:29:16',
 '2022-02-27 17:29:16',
 '2022-02-27 17:29:28',
 '2022-02-27 17:29:38',
 '2022-02-27 17:31:13',
 '2022-02-27 17:53:29',
 '2022-02-27 18:35:01',
 '2022-02-27 18:45:13',
 '2022-02-27 19:12:26',
 '2022-02-27 20:16:04']

In [73]:
rare_number_list[:10]

['undefined',
 '750830888173961351',
 '750830900253556981',
 '750830912618365258',
 '750831012895785085',
 '750832448371163361',
 '750835125108867084',
 '750835781550997624',
 '750837535575376122',
 '750841634400239661']

In [74]:
check_reply_list[:10]

[nan, '답글', '답글', '답글', nan, nan, nan, nan, nan, nan]

### 8페이지

In [594]:
# 8 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[8]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/958989023.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[8]').click()


In [595]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [596]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [597]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  401
고유번호 리스트 길이 :  401
답글구분 리스트 길이 :  401
닉네임 리스트 길이 :  401
작성자URL 리스트 길이 :  401
공감수 리스트 길이 :  401
링크수 리스트 길이 :  401
답글수 리스트 길이 :  401
첨부이미지URL 리스트 길이 :  401
내용 리스트 길이 :  401


### 9페이지

In [598]:
# 9 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[9]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/2827278859.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[9]').click()


In [599]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [600]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [601]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  451
고유번호 리스트 길이 :  451
답글구분 리스트 길이 :  451
닉네임 리스트 길이 :  451
작성자URL 리스트 길이 :  451
공감수 리스트 길이 :  451
링크수 리스트 길이 :  451
답글수 리스트 길이 :  451
첨부이미지URL 리스트 길이 :  451
내용 리스트 길이 :  451


### 10페이지

In [602]:
# 10 페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[10]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/3912001304.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[10]').click()


In [603]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [604]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [605]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  501
고유번호 리스트 길이 :  501
답글구분 리스트 길이 :  501
닉네임 리스트 길이 :  501
작성자URL 리스트 길이 :  501
공감수 리스트 길이 :  501
링크수 리스트 길이 :  501
답글수 리스트 길이 :  501
첨부이미지URL 리스트 길이 :  501
내용 리스트 길이 :  501


### 다음페이지(11페이지)

In [606]:
# 다음(11)페이지 이동
driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[11]').click()
time.sleep( random.uniform(1, 1.5) )

C:\Users\kbjung\AppData\Local\Temp/ipykernel_5300/4106609553.py:2: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="naverComment_201_222657505998"]/div/div[4]/div/a[11]').click()


In [607]:
# 페이지 수프에 담기
comment_page_soup = bs(driver.page_source, 'lxml')
comment_areas = comment_page_soup.find_all('li', 'u_cbox_comment')
len(comment_areas)

50

In [608]:
# 작성일 수집
for one in comment_areas:
    date = one.find('div', 'u_cbox_comment_box').find('span','u_cbox_date').attrs['data-value'].split('+')[0].replace('T',' ')
    date_list.append(date)

# 고유번호
for one in comment_areas:
    temp = one.attrs['class'][1].split('_')[-1]
    rare_number_list.append(temp)

# 답글 구분
for one in comment_areas:
    try:
        one.find('span', 'u_cbox_secret_contents').text
        temp = '비밀댓글'
    except:
        try:
            temp = one.find('div', 'u_cbox_reply_area').find('li')
            temp = nan
        except:
            temp = '답글'
    check_reply_list.append(temp)

# 닉네임
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_nick').text
    except:
        temp = '비밀댓글'
    nickname_list.append(temp)

# 작성자 url
for one in comment_areas:
    try:
        temp = one.find('a', 'u_cbox_name').attrs['href']
    except:
        temp = '비밀댓글'
    writer_url_list.append(temp)

# 공감수
for one in comment_areas:
    try:
        temp = int( one.find('em', 'u_cbox_cnt_recomm').text )
    except:
        temp = 0
    like_num_list.append(temp)

# 링크 수
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').find_all('a', 'u_cbox_contents_link')
        temp = len(temp)
    except:
        temp = 0
    link_num_list.append(temp)

# 답글 수
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_reply_area').find_all('li')
        temp = len(temp)
    except:
        temp = 0
    reply_num_list.append(temp)

# 첨부이미지url
for one in comment_areas:
    try:
        temp = one.find('div', 'u_cbox_image_section').find('img').attrs['src']
    except:
        temp = nan
    img_list.append(temp)

# 댓글 내용 수집
for one in comment_areas:
    try:
        temp = one.find('span', 'u_cbox_contents').text.replace('@', ' @', 1).replace('http', ' http', 1)
    except:
        temp = '비밀댓글'
    comment_list.append(temp)

In [609]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  551
고유번호 리스트 길이 :  551
답글구분 리스트 길이 :  551
닉네임 리스트 길이 :  551
작성자URL 리스트 길이 :  551
공감수 리스트 길이 :  551
링크수 리스트 길이 :  551
답글수 리스트 길이 :  551
첨부이미지URL 리스트 길이 :  551
내용 리스트 길이 :  551


### 12페이지

## 수집한 데이터 확인

In [626]:
print('작성일 리스트 길이 : ', len(date_list))
print('고유번호 리스트 길이 : ', len(rare_number_list))
print('답글구분 리스트 길이 : ', len(check_reply_list))
print('닉네임 리스트 길이 : ', len(nickname_list))
print('작성자URL 리스트 길이 : ', len(writer_url_list))
print('공감수 리스트 길이 : ', len(like_num_list))
print('링크수 리스트 길이 : ', len(link_num_list))
print('답글수 리스트 길이 : ', len(reply_num_list))
print('첨부이미지URL 리스트 길이 : ', len(img_list))
print('내용 리스트 길이 : ', len(comment_list))

작성일 리스트 길이 :  731
고유번호 리스트 길이 :  731
답글구분 리스트 길이 :  731
닉네임 리스트 길이 :  731
작성자URL 리스트 길이 :  731
공감수 리스트 길이 :  731
링크수 리스트 길이 :  731
답글수 리스트 길이 :  731
첨부이미지URL 리스트 길이 :  731
내용 리스트 길이 :  731


## df 만들기

In [627]:
dict = {'작성일':date_list, '고유번호':rare_number_list, 
        '답글구분':check_reply_list, '닉네임':nickname_list, 
        '작성자URL':writer_url_list, '공감수':like_num_list, 
        '링크수':link_num_list, '답글수':reply_num_list, 
        '첨부이미지URL':img_list, '내용':comment_list}
df = pd.DataFrame(dict)

In [628]:
df.head()

,작성일,고유번호,답글구분,닉네임,작성자URL,공감수,링크수,답글수,첨부이미지URL,내용
0,2022-02-25 12:58:42,750627914109682094,NaN,문득,https://blog.naver.com/ekdlskdl,0,0,0,NaN,*정답: 3번 @해피쏭
1,2022-02-25 12:59:29,750627965884170523,NaN,푸른하늘현,https://blog.naver.com/blueday118,0,1,0,NaN,정답 : 3번 https://blog.naver.com/blueday118/2226...
2,2022-02-25 13:05:52,750628376070324774,NaN,란,https://blog.naver.com/cdism,0,1,0,NaN,*정답: 3번 https://blog.naver.com/cdism @란 @cdism
3,2022-02-25 13:07:04,750628453866274870,NaN,saypark3824,https://blog.naver.com/saypark3824,0,0,0,https://g-cbox.pstatic.net/MjAyMjAyMjVfNTkg/MD...,정답 : 3번 입니다 @최준원 학생기자
4,2022-02-25 13:07:35,750628486162415983,NaN,리체,https://blog.naver.com/litvice,0,0,0,NaN,3번~~~^^


## 파일로 출력

In [629]:
crawled_folder_path

'c:/data/Wantreez/Crawling/incheon/crawled_data/naver_blog'

In [630]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
today_date

'20220415_133755'

In [631]:
today_date = datetime.today().strftime('%Y%m%d_%H%M%S')
file_name = f'naver_blog_{today_date}.xlsx'
df.to_excel(crawled_folder_path + '/' + file_name, index=False, encoding='utf-8')
print(f'{file_name} 출력완료')

naver_blog_20220415_133755.xlsx 출력완료


# 코드 마지막